In [ ]:
# example of loading the keras facenet model
from keras.models import load_model
from mtcnn.mtcnn import MTCNN
import cv2
# load the model
model = load_model('models/facenet_keras.h5')
img_path = "test images/lena.jpg"

def loadImg(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

detector = MTCNN()

In [ ]:
import numpy as np
import time
np.set_printoptions(suppress=True)

def drawFaceRectangles(image, results):
    img = np.copy(image)
    for i in range(len(results)):
        x1, y1, w, h = results[i]['box']
        cv2.rectangle(img, (x1, y1), (x1+w, y1+h), color=(0, 255, 0)) #Color in BGR
    return img

def extractFaces(image, results):
    faces = np.zeros((0, 160, 160, 3), dtype=np.uint8)
    for i in range(len(results)):
        x1, y1, w, h = results[i]['box']
        face = image[y1:(y1+h), x1:(x1+w)]
        face = cv2.resize(face, (160, 160))
        faces = np.append(faces, face.reshape(1, 160, 160, 3), axis=0)
    return faces

def normalizeImage(image):
    mean, std = image.mean(), image.std()
    image = (image - mean) / std
    return image

image = loadImg(img_path)
strt = time.time()
results = detector.detect_faces(image)
print(time.time()-strt)

faces_arr = extractFaces(image, results)
normalized_face = normalizeImage(faces_arr[0])
embeddings = model.predict(normalized_face.reshape(1, 160, 160, 3))
print(embeddings)
cv2.imshow("Detections", faces_arr[0])
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
def l2_distance(arr1, arr2):
    diff = arr1 - arr2
    return np.sqrt(np.sum(diff*diff))

def cosine_distance(arr1, arr2):
    scalar_product = np.sum(arr1*arr2)
    norm1 = np.sqrt(np.sum(arr1*arr1))
    norm2 = np.sqrt(np.sum(arr2*arr2))
    return scalar_product/(norm1*norm2)

img1 = loadImg("test images/luke1.jpg")
img2 = loadImg("test images/mitin2.jpg")
img3 = loadImg("test images/mitin1.jpg")

res1 = detector.detect_faces(img1)
res2 = detector.detect_faces(img2)
res3 = detector.detect_faces(img3)

face1 = extractFaces(img1, res1)
face2 = extractFaces(img2, res2)
face3 = extractFaces(img3, res3)

emb1 = model.predict(normalizeImage(face1))
emb2 = model.predict(normalizeImage(face2))
emb3 = model.predict(normalizeImage(face3))

print("Distance between 1 and 2: ", l2_distance(emb1, emb2))
print("Distance between 1 and 3: ", l2_distance(emb1, emb3))
print("Distance between 2 and 3: ", l2_distance(emb2, emb3))

print("Cosine distance between 1 and 2: ", cosine_distance(emb1, emb2))
print("Cosine distance between 1 and 3: ", cosine_distance(emb1, emb3))
print("Cosine distance between 2 and 3: ", cosine_distance(emb2, emb3))

print("norm1: ", np.sqrt(np.sum(emb1*emb1)))
print("norm2: ", np.sqrt(np.sum(emb2*emb2)))
print("norm3: ", np.sqrt(np.sum(emb3*emb3)))

In [ ]:
from sklearn.svm import SVC
import pickle

X = np.append(emb1, emb2, axis=0)
y = np.array([0, 1])
print(X.shape)

clf = SVC(gamma='auto')
clf.fit(X, y)
print(clf.predict(emb3))

with open('classifier.pickle', 'wb') as f:
     pickle.dump(clf, f)
        
with open('classifier.pickle', 'rb') as f:
     clf2 = pickle.load(f)
        
print(clf2.predict(emb3)) 